In [1]:
#Data Manipulation packages
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize 

#webscraping packages
from bs4 import BeautifulSoup as BS
import requests
import json
#import geocoder #Using the geocoder to find long and lat
#from geopy.geocoders import Nominatim

#Machine Learning Packages
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans # All packages related to Kmeans and the evaluation
from sklearn import metrics

#Visualization Packages
import matplotlib.pyplot as plt
!conda install -c conda-forge folium=0.5.0 --yes
import folium# Used for creating interactive maps
import plotly.plotly as py #preferred package to create interactive plots and maps
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#declaring this. Much similar to %matplotlib inline.
init_notebook_mode(connected=True)


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [2]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data

--2019-02-01 03:17:10--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-02-01 03:17:11--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-01 03:17:11--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-01 03:17:12--  https://ibm.ent.box.com/shared/static

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

postal =requests.get(url)
html = postal.text

soup = BS(html, 'lxml')

In [4]:
soup_pret = soup.prettify()

table = soup.find('table')

In [5]:
Post = []
Borough = []
Neighborhood = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        Post.append(cells[0].get_text())
        Borough.append(cells[1].get_text())
        Neighborhood.append(cells[2].get_text())

print(Post[:10])
print(Borough[:10])
print(Neighborhood[:10])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A', 'M8A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'North York', "Queen's Park", 'Not assigned']
['Not assigned\n', 'Not assigned\n', 'Parkwoods\n', 'Victoria Village\n', 'Harbourfront\n', 'Regent Park\n', 'Lawrence Heights\n', 'Lawrence Manor\n', 'Not assigned\n', 'Not assigned\n']


In [6]:
neigh = []

for i in Neighborhood:
    index = i.find("\n")
    neigh.append(i[:index].rstrip())

print(neigh[:10])

['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront', 'Regent Park', 'Lawrence Heights', 'Lawrence Manor', 'Not assigned', 'Not assigned']


In [7]:
df = pd.DataFrame(Post, columns = ['Postcode'])
df['Borough'] = Borough
df['Neighborhood'] = neigh

In [8]:
df1 = df[df['Borough'] != 'Not assigned'].reset_index(drop = True)

df1 = df1.copy()

df1.loc[6, 'Neighborhood'] = "Queen's Park"

df1

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
